# **Disclaimer:**
This assignment must be completed solely by the members of your group. Sharing of code between groups is strictly prohibited. However, you are allowed to discuss general solution approaches or share publicly available resources with members of other groups. Therefore, clearly indicate which public resources you consulted and/or copied code from. Any plagiarism between groups will result in the initiation of a fraud procedure with the director of education.

# **Introduction**

In this assignment, you will train a Reinforcement Learning (RL) agent to address a Residential Energy Management problem.

The goal is for your agent to control a battery and optimize the electricity costs for a residential household. You will submit your trained agent, which will first be evaluated using an evaluation dataset (different from what you have). Following that, your agent will be deployed in a real-world setting: the [HomeLab](https://idlab.ugent.be/resources/homelab). After deployment, you will receive data detailing the performance of your agent in both environments. Finally, you will write a brief essay analyzing and discussing the results. \
Moreover, we wrote a set of questions / tasks you can fill in the notebook. Each question will contribute to the final grade you will obtain from this assignment.

The task involves managing energy for a residential household equipped with a Photovoltaic (PV) system and an Energy Storage System (ESS), specifically a battery. In the context of the global shift toward decarbonizing energy sources, these types of challenges are becoming increasingly significant. Reinforcement Learning (RL) has proven to be a promising technique for addressing them.

The setup includes a household where residents consume electricity daily, resulting in a unique consumption profile. Electricity costs are based on a dynamic price profile, with prices varying by the hour. When the household generates more energy (from the PV system or battery) than it consumes, the excess will be sold back to the grid, although the selling price is typically much lower than the buying price.

Your objective is to train an RL agent to control the battery and minimize electricity costs. While this might initially seem straightforward (e.g., charge the battery when prices are low and discharge it when prices are high), there are several challenges that complicate the problem:

*   **Uncertainty in household consumption**: Future consumption is difficult to predict. While models based on inhabitants' habits can provide reasonable estimates[1], there is always a stochastic element involved.
*   **Variability in PV production**: PV energy generation depends on weather conditions, adding another layer of unpredictability.
*   **Dynamic price profiles**: Although, for simplicity, we assume a single price profile repeats daily in this assignment, real-world scenarios often involve prices that vary from day to day.
*   **Physical constraints**: The control of the battery and the accuracy of data from the household (such as PV production and overall consumption) are not perfect. While these issues are ignored in the simulation, they will become evident in the real-world results from HomeLab after your agent has been deployed.

Additionally, although not considered in this assignment, more complex setups may include assets like heat pumps or electric vehicle chargers. These cases may require an extended RL framework, potentially involving multi-agent systems.

In the assignment, you will use an agent based on the [PPO](https://arxiv.org/pdf/1707.06347) algorithm. There are some libraries that have already implemented the algorithm, the most famous being [stable-baselines3](https://stable-baselines3.readthedocs.io/en/master/#). However, in this assignment, you will write the algorithm code yourself. To help you with the task, we provide you with most of the code you will need to develop a PPO agent.

\

---

\

[1] e.g., by observing that they usually leave for work from 9:00 AM to 18:00 PM and cook at 19:00 PM, we can obtain a sensible model of the expected consumption

\

---

\


Let's now delve into the assignment details!

# **Support Classes**

In this assignment, we provide the essential code to help you train a PPO agent for a Home Energy Management (HEM) problem. To begin, download the material.zip file available in the course resources. This file contains the necessary code and information to complete the assignment.

After downloading, extract the .py files and upload them into the 'Files' section of this Colab notebook. Next, run the code snippet below, which will import all required external libraries, along with the support classes from the files you just uploaded.

We advise you **not** to change the code we provided you in such files. Any changes in those files that hinder our evaluation of your agents might result in your submission receiving a lower grade.

**N.B:** For training the neural networks used in the PPO algorithm, we require that you use the PyTorch library included in the provided files. Using other deep learning libraries (e.g., TensorFlow) could cause compatibility issues when evaluating your agent.

If you require additional standard Python packages for your work, feel free to import them in the following cell. (by standard Python packages we consider the ones included in the Google colab environment. Avoid using any 'pip install' call in this notebook) \
In case you want to download the notebook and run it locally, you can recreate the colab environment by installing the same versions in your local machine. The list of packages' versions can be seen through the command `!pip freeze` executed below.

In [1]:
import sys
import numpy as np
import torch
from torch import nn
from torch.distributions import MultivariateNormal
import matplotlib.pyplot as plt
import pickle

sys.path.insert(0, 'Environment.py')
sys.path.insert(0, 'nn_utils.py')

from Environment import BaseEnvironment
from nn_utils import GenericNeuralNetwork



# Extra imports:
# import ...

In [2]:
!pip freeze

anyio==4.11.0
argon2-cffi==25.1.0
argon2-cffi-bindings==25.1.0
arrow==1.4.0
asttokens==3.0.0
async-lru==2.0.5
attrs==25.4.0
babel==2.17.0
beautifulsoup4==4.14.2
bleach==6.2.0
certifi==2025.10.5
cffi==2.0.0
charset-normalizer==3.4.4
comm==0.2.3
contourpy==1.3.3
cycler==0.12.1
debugpy==1.8.17
decorator==5.2.1
defusedxml==0.7.1
executing==2.2.1
fastjsonschema==2.21.2
filelock==3.20.0
fonttools==4.60.1
fqdn==1.5.1
fsspec==2025.10.0
h11==0.16.0
httpcore==1.0.9
httpx==0.28.1
idna==3.11
ipykernel==7.0.1
ipython==9.6.0
ipython_pygments_lexers==1.1.1
isoduration==20.11.0
jedi==0.19.2
Jinja2==3.1.6
json5==0.12.1
jsonpointer==3.0.0
jsonschema==4.25.1
jsonschema-specifications==2025.9.1
jupyter-events==0.12.0
jupyter-lsp==2.3.0
jupyter_client==8.6.3
jupyter_core==5.9.1
jupyter_server==2.17.0
jupyter_server_terminals==0.5.3
jupyterlab==4.4.9
jupyterlab_pygments==0.3.0
jupyterlab_server==2.27.3
kiwisolver==1.4.9
lark==1.3.0
MarkupSafe==3.0.3
matplotlib==3.10.7
matplotlib-inline==0.1.7
mistune==3.1.4

# **Environment**

Now that you imported the classes from the files we provided you, you can start building an RL framework. RL agents learn how to obtain optimal actions by interacting with an environment, modeled by a Markovian Decision Process. The class 'BaseEnvironment' contained in the file 'Environment.py' is exactly that.

Let's initialize a BaseEnvironment class:

First, you need to fix some hyperparameters defining the details of the environment.
- `power_capacity` is the max. power the battery is physically capable of injecting (or withdrawing). It is expressed in kW
- `energy_capacity` is the max. energy the battery is physically allowed to store. It is expressed in kWh

The battery used in the evaluations has a power capacity of 4kW and an energy capacity of 8kWh. Therefore, the agent you upload should be trained with these capacity values.

You will also need to load the 'public_data_dict.pkl' we provided you. The loaded file is a dictionary containing historical data on PV production and household consumption for the weekdays of almost a whole year. Every instance of the `BaseEnvironment` requires as input a data dictionary with the same format we provided you with. \
Further details of the environment are contained in the documentation we provided in the .zip file. **Please read it carefully**

In [3]:
# HYPERPARAMETERS:
power_capacity = 4
energy_capacity = 8

### Environment initialization
with open('public_data_dict.pkl', 'rb') as f:
    data_dict = pickle.load(f)

env = BaseEnvironment(power_capacity = power_capacity,
                      energy_capacity = energy_capacity,
                      data = data_dict)


In machine learning, it is common practice to divide a dataset into at least two parts:

- A training set, used to train the model;
- A validation set, which remains completely unseen by the model during training and is used to assess its performance during and after training.

This division is not always allowed in RL. For example, in board game environments such as chess you do not necessarily have historical data to divide. Hence, in cases like that it is not possible to separate into different sets. \
However, in problems like this where historical data is available, you can (and should) divide sets to ensure the agent is not simply overfitting on the training set. Below we suggest a possible division that is based on the data available and the conditions in which the agent will be evaluated.

Additionally, the policy of an agent during training often differs from its policy during evaluation. For instance, when evaluating an agent, it may be beneficial to minimize the effect of stochastic events by standardizing evaluation conditions. One way to achieve this is to consistently use the agent's greedy actions during evaluation and ensure that the environment always starts with the same initial states.

In [4]:
train_length = int(len(data_dict) * 0.8)

train_set_indexes = list(range(0, train_length))
test_set_indexes = list(range(train_length, len(data_dict)))

# **PPO Agent**

Now, it is time to start writing the PPO code. In the .zip file, we provided you with some utility classes that will help you train and use a PyTorch neural network. \
Moreover, to help you with the task of implementing a PPO algorithm, we provided you with most of the code structure. \
Below, you can find the class of the PPO agent you will use in the assignment. First, we ask you to **carefully read the code** and understand it as much as possible. The code misses some parts that you will need to fill in. In particular, you need to complete the following methods:
- `train`: Perform `num_iterations` training steps. In each training step, perform `num_batches` rollouts and train both the actor and the critic on the obtained data. In this method, 2 portions needs to be completed. First, you will need to calculate the estimated advantage values `A_k`. For stability reasons, it is good practice to normalize these values by scaling them into a more manageable interval (e.g., ~[-1, 1]); how can you do so? If you do not scale the estimations, what do you notice in your training curve? Then, you need to implement the PPO loss for the actor. For that, it can be handy to use the torch function `torch.clamp`.
- `calculate_value_logprobs`: Given the batched states and actions, it returns the estimated values of the batched states (from the critic network) and the logarithmic probabilities of the batched actions in relation to the current actor distribution. Check the documentation of the `MultivariateNormal` class, as it will be quite useful for implementing this function.
- `sample_action`: Given the input `state`, it returns the corresponding action from the actor, and the logarithmic probability of the action in relation to the current actor distribution. There is also a boolean input `explore`. If the boolean is set to `True`, the output is an action sampled from the actor probability distribution. Else, if the boolean is set to `False`, the action will be the most likely one from the actor probability distribution. Check the documentation of the `MultivariateNormal` class, as it will be quite useful for implementing this function.
- `rollout`: This function perform `num_batches` rollout. Each rollout correspond to the execution of one episode sampling from the actor policy distribution. You can use the `sample_action` method for that. Part of the rollout loop is already provided. You will need to complete it by properly filling the batch lists (`batch_action`, `batch_states`, `batch_rewards`, and `batch_log_probs`) with their corresponding values.

**Important: The agent and critic should be working and training with normalized (i.e., scaled) states, actions, and rewards. Make sure to implement the following code sections accordingly. When evaluating your agent, we will assume your actor neural network takes the normalized state as input and returns the normalized actions. We will normalize the states and de-normalize the actions accordingly to the environment documentation.**

In [30]:
class PPOHyperparameters():

    def __init__(self, num_batches: int, sampling_variance: float, gamma: float, updates_per_iteration: int, clip_value: float, actor_lr: float, critic_lr: float, ):
        self.num_batches = num_batches

        # Variance of the multivariate Normal distribution used to sample exploratory actions
        self.sampling_variance = sampling_variance

        # Discount factor
        self.gamma = gamma

        # Number of updates in each training iteration
        self.updates_per_iteration = updates_per_iteration

        # Epsilon used in the clip for training
        self.clip_value = clip_value

        # Networks Learning Rate
        self.actor_lr = actor_lr
        self.critic_lr = critic_lr

class PPOAgent:

    def __init__(self, env, state_dim, action_dim, actor: nn.Module, critic: nn.Module, hyperparameters: PPOHyperparameters):

        self.env = env

        self.state_dim = state_dim
        self.action_dim = action_dim

        self.hyperparameters = hyperparameters

        self.actor = actor
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=self.hyperparameters.actor_lr)

        self.critic = critic
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=self.hyperparameters.critic_lr)

        self.cov_var = torch.full(size=(self.action_dim,), fill_value = self.hyperparameters.sampling_variance)
        self.cov_mat = torch.diag(self.cov_var) # This will be used to sample actions from the agent distribution. The higher the variance, the wider the gaussian distribution.


    def train(self, num_iterations:int = 200):
        for iteration in range(num_iterations):
            batch_states, batch_actions, batch_log_probs, batch_cumulative_rewards, batch_rewards = self.rollout()

            V, _ = self.calculate_value_logprobs(batch_states=batch_states, batch_actions=batch_actions)
            V = V.detach()

            # Exercise: Calculate the Advantage and normalize it
            # A_k = ...
            advantages = batch_cumulative_rewards - V
            # normalize
            advantages = (advantages - advantages.mean()) / (advantages.std(unbiased=False) + 1e-8)

            for _ in range(self.hyperparameters.updates_per_iteration):
                current_V, current_log_probs = self.calculate_value_logprobs(batch_states=batch_states, batch_actions=batch_actions)

                probability_ratios = torch.exp(current_log_probs - batch_log_probs)

                # Exercise: Implement the PPO loss from the probability_ratios variable
                # actor_loss = ...
                clip_eps = self.hyperparameters.clip_value
                unclipped_obj = probability_ratios * advantages
                clipped_ratios = torch.clamp(probability_ratios, 1.0 - clip_eps, 1.0 + clip_eps)
                clipped_obj = clipped_ratios * advantages

                # we minimize loss, so take negative of objective and mean
                actor_loss = -torch.mean(torch.min(unclipped_obj, clipped_obj))

                
                self.actor_optimizer.zero_grad()
                actor_loss.backward(retain_graph=True)
                self.actor_optimizer.step()

                critic_loss = torch.nn.MSELoss()(current_V, batch_cumulative_rewards.squeeze(-1))
                self.critic_optimizer.zero_grad()
                critic_loss.backward()
                self.critic_optimizer.step()


    def calculate_value_logprobs(self, batch_states, batch_actions):
        """
        batch_states: tensor (N, state_dim) -- normalized
        batch_actions: tensor (N, action_dim) or (N,) but in training you used unsqueeze so it's shape (N,1)
        returns: V (N,), log_probs (N,)
        """
        # Value estimates
        V = self.critic(batch_states).squeeze(-1)  # shape (N,)

        # Actor predicted mean for actions (normalized actions)
        action_means = self.actor(batch_states)  # shape (N, action_dim)

        # Ensure batch_actions is shaped (N, action_dim)
        if batch_actions.dim() == 1:
            actions = batch_actions.unsqueeze(-1)
        else:
            actions = batch_actions
        actions = actions.view(action_means.shape)  # align shapes

        # Create multivariate normal with diagonal covariance (same for all samples)
        cov_mat = self.cov_mat.to(action_means.device)
        dist = MultivariateNormal(loc=action_means, covariance_matrix=cov_mat)

        # log_prob returns (N,) for multivariate
        log_probs = dist.log_prob(actions)

        return V, log_probs

        
    def sample_action(self, state, explore=False):
        """
        state: raw environment state (list/array)
        explore: if True sample from distribution; otherwise return mean (deterministic)
        returns:
            action (numpy scalar or array) -> normalized action (so actor/critic are trained on normalized actions)
            log_prob (torch.tensor scalar)
        """
        # Normalize state for the actor
        norm_state = np.array(self.env.scale_state(state), dtype=np.float32)
        state_tensor = torch.tensor(norm_state, dtype=torch.float).unsqueeze(0)  # shape (1, state_dim)

        # Actor mean (normalized action)
        with torch.no_grad():
            action_mean = self.actor(state_tensor).squeeze(0)  # shape (action_dim,)

        cov_mat = self.cov_mat.to(action_mean.device)
        dist = MultivariateNormal(loc=action_mean, covariance_matrix=cov_mat)

        if explore:
            sampled_action = dist.sample()  # tensor (action_dim,)
        else:
            sampled_action = action_mean  # deterministic

        log_prob = dist.log_prob(sampled_action)  # scalar tensor

        # Return numpy for convenience and the log_prob tensor
        return sampled_action.cpu().numpy(), log_prob


    def compute_cumulative_rewards(self, batch_rewards):
        batch_cumulative_rewards = []

        for reward_episodes in batch_rewards:  # iterate normally
            discounted_reward = 0
            for reward in reversed(reward_episodes):
                discounted_reward = self.hyperparameters.gamma * discounted_reward + reward
                batch_cumulative_rewards.insert(0, discounted_reward)  # insert in front

        # Flatten in case some inner lists got in
        batch_cumulative_rewards = [float(r.item()) if isinstance(r, np.ndarray) else float(r) for r in batch_cumulative_rewards]

        batch_cumulative_rewards = torch.tensor(batch_cumulative_rewards, dtype=torch.float)
        return batch_cumulative_rewards


    def rollout(self):
        # NOTE: Put the correct sizes as comment here
        batch_states = []       # (num_batches * timesteps_per_episode, state_dim)
        batch_actions = []      # (num_batches * timesteps_per_episode, action_dim)
        batch_log_probs = []    # (num_batches * timesteps_per_episode)
        batch_rewards = []      # (num_batches, timesteps_per_episode)

        for t in range(self.hyperparameters.num_batches):
            self.env.reset(np.random.choice(train_set_indexes))   # Train on a random day from the training set
            rewards_episode = []

            state = self.env.state
            done = self.env.done

            while not done:
                # Sample normalized action from policy
                norm_action, log_prob = self.sample_action(state, explore=True)

                # Convert normalized → real-world action
                real_action = self.env.descale_action(norm_action)

                # Step in environment
                next_state, reward, done, _ = self.env.step(real_action)

                # Normalize state
                norm_state = self.env.scale_state(state)
                batch_states.append(norm_state)

                # Store normalized action (float or array)
                batch_actions.append(
                    float(norm_action) if np.isscalar(norm_action) or np.shape(norm_action) == ()
                    else np.array(norm_action, dtype=float)
                )

                # Store log probability
                batch_log_probs.append(log_prob.detach().cpu().item())

                # Scale reward for stability
                scaled_reward = self.env.scale_reward(reward)
                rewards_episode.append(scaled_reward)

                state = next_state

            batch_rewards.append(rewards_episode)

        # Convert arrays for tensors
        batch_states = torch.tensor(np.array(batch_states), dtype=torch.float)
        batch_actions_array = np.array(batch_actions)  # or np.stack(batch_actions) if shapes match
        batch_actions_tensor = torch.tensor(batch_actions_array, dtype=torch.float).unsqueeze(dim=-1)
        batch_log_probs = torch.tensor(batch_log_probs, dtype=torch.float)

        # ✅ DO NOT CONVERT batch_rewards TO TENSOR HERE
        batch_cumulative_rewards = self.compute_cumulative_rewards(batch_rewards=batch_rewards)

        print(type(batch_rewards), len(batch_rewards), type(batch_rewards[0]))

        return batch_states, batch_actions_tensor, batch_log_probs, batch_cumulative_rewards, batch_rewards




    def save_agent(self):
        with open('actor.pickle', 'wb') as handle:
            pickle.dump(self.actor, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def load_agent(self):
        with open('actor.pickle', 'rb') as handle:
            self.actor = pickle.load(handle)

# **Interaction agent-environment**

Now that you’ve written the code for the PPO agent, it’s good practice to perform some sanity checks to ensure the agent behaves as expected. But first, we need to implement the code that will serve as the interface between the environment and the agent.

Let’s begin by initializing the agent class. Below is an example of hyperparameters you can use. These values are **intentionally** chosen to be suboptimal. Meaning that the code will run with those values, but the agent **will not have good training performance**. \
**Remember to adjust them when you start evaluating the agent's performance.**

In [31]:
state_dim = 5       # Dimension of each state, do not change
action_dim = 1      # Dimension of the actions, do not change

actor_nn_params = dict(input_size=state_dim,
                       layers=[(4096, 'relu', 0.9), # (number_of_neurons, activation_function, dropout_rate)
                               (2048, 'relu', 0.9), ],
                       output_size=action_dim,
                       activation_final='tanh')
actor = GenericNeuralNetwork(params=actor_nn_params)

critic_nn_params = dict(input_size=state_dim,
                        layers=[(2048, 'relu', 0.9),
                                (4096, 'relu', 0.9), ],
                        output_size=1,
                        activation_final='linear')
critic = GenericNeuralNetwork(params=critic_nn_params)

hyperparameters = PPOHyperparameters(num_batches=1,                 # Number of episodes per rollout
                                     sampling_variance=5.735,       # Variance of the actor gaussian distribution
                                     gamma=0.25,                    # Discount factor
                                     updates_per_iteration=19345,   # Neural Network backpropagation steps per train() call
                                     clip_value=0.9,                # Clip value from the PPO loss
                                     actor_lr=30.857,               # Learning rate of the Actor
                                     critic_lr=5.12)                # Learning rate of the Critic

ppo_agent = PPOAgent(env=env, state_dim=state_dim, action_dim=action_dim, actor=actor, critic=critic, hyperparameters=hyperparameters)

# **Training loop**

Finally, we are ready to train and evaluate the agents. To do so, you can use the train method of the PPO agent. It is good practice to regularly evaluate the agent after a certain amount of iteration during the training phase. In the evaluation process, the actions applied to the environment should be the greedy  ones (i.e., the action the agent currently believes to be the optimal). We provided you with the structure of an evaluation method, you can fill the missing code and properly use it in your training phase \
As you experiment with this assignment, think of how you can improve the training based on the knowledge you learned in the classes.

In [ ]:
def environment_episode(day_index: int = 0):
    '''
    Reset and interact with the environment until the end of the selected day.

    day_index: Code of the day used in the environment
    '''
    ppo_agent.env.reset(day_index)
    total_reward = 0

    while not env.done:
        state = ppo_agent.env.state
        action, _ = ppo_agent.sample_action(state, explore=False)
        action = action.detach().item()
        next_state, reward, done, _ = env.step(ppo_agent.env.descale_action(action))

        total_reward += reward

    return total_reward

def evaluate_agent(num_eval_days: int = 10):
    """
    Runs deterministic evaluation episodes on different days and returns
    the mean total reward.
    """
    total_rewards = []

    # Choose distinct or random days from your training set
    for _ in range(num_eval_days):
        day_index = np.random.choice(train_set_indexes)
        total_reward = environment_episode(day_index)
        total_rewards.append(total_reward)

    # Compute the average total reward
    evaluation_reward = np.mean(total_rewards)
    return evaluation_reward

num_iteration = 100
iteration_per_evaluation = 1

for training_iteration in range(1, num_iteration):
    print(training_iteration)
    ppo_agent.train(num_iterations=1)

    if (training_iteration % iteration_per_evaluation) == 0:
        reward = evaluate_agent()
        print(training_iteration, reward)

1
<class 'list'> 1 <class 'list'>


# **Conclusions**

Now you have everything you need to start training an effective agent.

To proceed further, you can change the hyperparameters, write new functions to keep track of the training process, and produce informative graphs that can help you understand what is going on behind the curtains of the model. \
To double check the code you just wrote, try to train and evaluate the agent on a single day (i.e., using the same data of 1 single day for both training and evaluation). This is a very simple task, hence your agent is supposed to behave well without too many difficulties. For example, try with day number 100. With that day, your agent should be able to achieve a daily total reward that is at least greater than -0.4 (when using a 4kW, 8kWh battery). \
Keep in mind that the algorithm will require some time to get trained. Don't give up on a parameter setting after only a bunch of minutes of training. Sometimes the agent might just need some extra episodes of training to start getting some sense of its actions.

**Your final agent needs to be saved using the methods we provided you. The agent you will upload will be evaluated on a private dataset taken out of the data we gave you, and the results will influence the score of your assignment.**

Whatever extra code you think you will need to train the agent, please write it below. \
**Moreover, we prepared a set of questions and tasks for you. These are part of your assignment evaluation.**

### **Question 0**

As explained before, we ask you to train your agent by working with normalize states, actions, and rewards by scaling their original value in more contained domain ([0, 1] or [-1, 1]). Can you reflect on this, giving your thoughts on why this should be (or should not be) done?

Write you brief considerations below: \
... \
...

### **Question 1**
Coming out with good hyperparameters is not straightforward. Can you make a list below of the **most important** hyperparameters you had to choose to obtain better results? Give brief intuitions for each. If needed, you can back it up with evaluation results.

- hyperparam1: fixed to x because of ...
- hyperparam2: fixed to y because of ...
- hyperparam3: fixed to z because of ...


### **Question 2:**
When training an RL agent (or whatever Machine Learning model), a crucial step is to track the learning. Can you produce some informative graphs that tells you how is the training going? What observation can you draw from those?

Can you think of some other insightful graphs / information about the agent? For example, what kind of extra metric can you keep track of? Here are some examples:
- Probability ratio of the actions during training
- KL divergence of the policy before and after each training step

Briefly explain what you did and why here:

... \
... \
...

In [ ]:
# Training graphs
# ...
# ...

### **Question 3:**

When using RL on a given problem, **you** also need to understand what the problem is about. Following that, it is important to keep track of the agent's performance by not only checking its numerical metrics (e.g., the obtained daily rewards), but also by checking if its actions make sense in the first place. To do so, plotting the policies obtained on a limited interval of time (e.g., a single day) is crucial. Can you write a function that does that? What observations / conclusions can you draw when you plot the agent policies?

Can you also think of additional way you can plot the policy without specifically referring to any specific day? In other words, can you generate a plot that describes the general behavior of the learnt policy in relation to some state inputs (not necessarily from the actual dataset; they could be artificially made just for the sake of the plot)?

Briefly explain your observations here:

... \
... \
...

In [ ]:
# Policy graphs
# ...
# ...

### **Question 4.1:**

PPO stands in the RL algorithm family of policy gradients methods. These differ from value-based methods such as Deep Q-Network (DQN) in the way the agent obtain the actions.
Policy gradient methods have an actor model that directly returns an action after processing an input state. In value-based methods, instead, the actor processes the input state to approximate the Q-function (or the value function) of the problem; the action is then selected based on the approximated Q-function values (for example by argmaxing each Q values). \
Specifically thinking about this problem (home energy management), can you reflect on what major advantages/disadvantages one algorithmic family brings over the other?

Briefly explain your observations here:

... \
... \
...

### **Question 4.2:**

In this assignment, you used 2 neural networks as models for both the critic and actor. This is a choice we made when designing this assignment, but it is not mandatory. Other regression architectures/models could have been used. Can you think of any specific reason why we would want to use a different model?  Write some considerations regarding this, with specific focus on the actor model.

Briefly explain your observations here:

... \
... \
...

### **Question 5:**

One of the benefits of implementing an algorithm from scratch (besides the obvious one of understanding more deeply what the algorithm is about) is the total control over the code you just wrote. This enables you to customize your code to, for example, gain informative insights regarding the algorithm dynamics.
There are a few things you can implement in your PPO code to potentially improve its performance. Here we give you some suggestions, but feel free to also check online sources to take additional inspiration.
- Entropy regularization
- Generalized Advantage Estimation (check [paper](https://arxiv.org/pdf/1506.02438))
- Learnable sampling variance
- Mini-batch partition fo the rollout data when training
- Learning Rate Annealing

Can you implement some of these additional features and report their value over the agent training phase? You do not have to implement them all, but even if you do, **please report only 1 or 2 of them**. \
Make a complete report of what differences you noticed using or not using the additional feature you implemented. Consider it as a mini scientific report.

... \
... \
... \

In [ ]:
# Implemented features report
# ...
# ...